In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers, callbacks, regularizers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [2]:
seed_value = 42

os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [3]:
model_version = 'v7.0'

In [4]:
with open('data_schema.json', 'r') as f:
    config = json.load(f)
    
spectro_cols = config['spectro_cols']
photo_cols = config['photometry_cols']
generic_cols = config['generic_cols']
target_col = config['target_col']

In [5]:
df = pd.read_csv(f"data/processed/spectrometry_photometry.csv")

X = df[spectro_cols + photo_cols + generic_cols]
y = df[target_col]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed_value)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
def build_model(units=32, dropout_rate=0.2, l2_reg=0.01, patience=10, lr=0.001):
    input_shape = X_train_scaled.shape[1]
    
    optimizer = Adam(learning_rate=lr)
    
    inputs = keras.Input(shape=(input_shape,))
    
    x = layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(inputs)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(units * 2, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(units * 4, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(units * 8, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    outputs = layers.Dense(1)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae', 'mse', 'mape'])
    
    return model

In [9]:
class SavingKerasRegressor(KerasRegressor):
    def fit(self, X, y, **kwargs):
        # Extract current params
        current_params = self.get_params()
        
        # Construct unique filename from current parameters
        filename = "spec_photo_{}_units{}_do{}_l2{}_bs{}_ep{}_pat{}.h5".format(
            model_version,
            current_params.get('units'),
            current_params.get('dropout_rate'),
            current_params.get('l2_reg'),
            current_params.get('batch_size'),
            current_params.get('epochs'),
            current_params.get('patience'),
            current_params.get('validation_split')
        )
        os.makedirs(f"history/{model_version}", exist_ok=True)
        os.makedirs(f"models/{model_version}", exist_ok=True)
        model_path = os.path.join(f"models/{model_version}", f"model_{filename}.h5")
        history_path = os.path.join(f"history/{model_version}", f"history_{filename}.json")

        # Callbacks
        checkpoint_cb = ModelCheckpoint(
            model_path,
            save_best_only=True,
            save_format='h5',
            monitor='val_loss',
            mode='min',
            verbose=0
        )
        early_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Add callbacks and validation_split
        kwargs['callbacks'] = [checkpoint_cb, early_cb]
        kwargs['validation_split'] = current_params.get('validation_split')

        history = super().fit(X, y, **kwargs)
        
        # Save history to JSON
        with open(history_path, 'w') as f:
            json.dump(history.history, f)

        return history

In [10]:
regressor = SavingKerasRegressor(build_fn=build_model, verbose=0)

param_grid = {
    'units': [32, 64],
    'dropout_rate': [0.2, 0.3],
    'l2_reg': [0.001, 0.01],
    'lr': [0.00001, 0.0001, 0.001, 0.01],
    'patience': [20, 30, 50],
    'batch_size': [16, 32, 64],
    'epochs': [200, 500, 1000],
    'validation_split': [0.5, 0.6, 0.7]
}

C:\Users\Sahil\AppData\Local\Temp\ipykernel_35504\157220425.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  regressor = SavingKerasRegressor(build_fn=build_model, verbose=0)


In [11]:
grid = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=2, scoring='neg_mean_squared_error', verbose=2)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 2 folds for each of 2592 candidates, totalling 5184 fits
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, lr=1e-05, patience=20, units=32, validation_split=0.5; total time=57.9min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, lr=1e-05, patience=20, units=32, validation_split=0.5; total time=38.2min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, lr=1e-05, patience=20, units=32, validation_split=0.6; total time=55.0min


KeyboardInterrupt: 

In [12]:
print("Best Score: ", grid_result.best_score_)
print("Best Params: ", grid_result.best_params_)

NameError: name 'grid_result' is not defined